In [2]:
include("saltsolver.jl")
include("passive.jl")
myrand(n) = rand(n) - 0.5 + 1im * rand(n) - 0.5
symmetrized(f) = 0.5 * (f + ring_flip(f))

symmetrized (generic function with 1 method)

In [1]:
ℓ = 5
N = 4ℓ
L = 1.0
σ = 0.3
scl = 0.01
ɛr = 1 + scl*myrand(N)
ɛi = σ + scl*myrand(N)
F = 1 + scl*myrand(N)


5-element Array{Complex{Float64},1}:
  -0.21681+0.647494im 
 -0.646647+0.5408im   
 -0.843822+0.0728975im
  -0.56417+0.582443im 
 -0.153147+0.191338im 

In [6]:
println("Cnv, uniform everything")

N = 3
nsym = N
Lcav = 1.0
ℓ = 1
σ = 0.1
γ⟂ = 0.2
ωℓ = N * √(2*(1-cos(2π*ℓ/N)))
ωa = 4.0
D = 0.2
ɛ = (1+im*σ) * ones(N)
F = ones(N)
las = Laser(ɛ, F, ωa, γ⟂)

L = 1.0
h = L/N
Ering, ωring = ring_eigenpair(N, ℓ, h)
md = PassiveMode(copy(Ering), ωring)
laplacian!(J) = periodic!(J, h)
Dt = passive_threshold!(laplacian!, md, las, (1.0, 1.2))
ωt = real(md.ω)
Eplus = md.E / md.E[1]
Es = (Eplus, ring_flip(Eplus))

H, Is, Js, Ks, GD = overlap_integrals(Es, ωt, Dt, las, nsym, ℓ, Lcav);
I, J, K = Is[1], Js[1], Ks[1];
i, j, k = I/H, J/H, K/H;

Dt, imag(md.ω)

Cnv, uniform everything


(0.9332690479444662,9.3237349405047e-11)